In [ ]:
import os
import math
import time

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import LambdaLR
from torchvision import transforms, models

import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader

import csv

import random
import shutil
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Dataset, random_split
from torchvision import transforms, datasets
from PIL import Image
from transformers import ViTModel, ViTImageProcessor
import os
from tqdm import tqdm

In [ ]:
# Reducing Region_ID in the train and validation labels by 1

df = pd.read_csv('/data3/pratyush.jena/misc/MMT/SMAI_Project/Phase_2_data/Phase_2_data/labels_train.csv')  

df['Region_ID'] = df['Region_ID'] - 1

df.to_csv('/data3/pratyush.jena/misc/MMT/SMAI_Project/Phase_2_data/Phase_2_data/labels_train_new.csv', index=False)  


df = pd.read_csv('/data3/pratyush.jena/misc/MMT/SMAI_Project/Phase_2_data/Phase_2_data/labels_val.csv')  

df['Region_ID'] = df['Region_ID'] - 1

df.to_csv('/data3/pratyush.jena/misc/MMT/SMAI_Project/Phase_2_data/Phase_2_data/labels_val_new.csv', index=False)  

In [ ]:
train_csv = "/data3/pratyush.jena/misc/MMT/SMAI_Project/Phase_2_data/Phase_2_data/labels_train_new.csv"
train_img_dir = "/data3/pratyush.jena/misc/MMT/SMAI_Project/Phase_2_data/Phase_2_data/images_train"
val_csv = "/data3/pratyush.jena/misc/MMT/SMAI_Project/Phase_2_data/Phase_2_data/labels_val_new.csv"
val_img_dir = "/data3/pratyush.jena/misc/MMT/SMAI_Project/Phase_2_data/Phase_2_data/images_val"
num_classes = 15
batch_size = 4
epochs = 50
lr = 1e-4
weight_decay = 1e-2
warmup_epochs = 5
label_smoothing = 0.1
mixup_alpha = 0.2
num_workers = 2
save_path = "area.pth"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(42)
np.random.seed(42)

In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform

        self.filenames = self.data.iloc[:, 0].values

        self.labels = self.data.iloc[:, 5].values.astype(int) 

        assert self.labels.min() >= 0 and self.labels.max() < 15, "Label range error"

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.filenames[idx])
        image = Image.open(img_path).convert("RGB")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.4, 0.4, 0.4),
    transforms.RandAugment(num_ops=2, magnitude=9),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225]),
])
val_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225]),
])

train_dataset = CustomImageDataset(train_csv, train_img_dir, transform=train_transforms)
val_dataset = CustomImageDataset(val_csv, val_img_dir, transform=val_transforms)

train_loader = DataLoader(train_dataset, batch_size=batch_size,
                         shuffle=True, num_workers=num_workers,
                         pin_memory=True)

val_loader = DataLoader(val_dataset, batch_size=batch_size,
                         shuffle=False, num_workers=num_workers,
                         pin_memory=True)


model = models.convnext_base(pretrained=True)
in_features = model.classifier[2].in_features
model.classifier[2] = nn.Linear(in_features, num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss(label_smoothing=label_smoothing)
optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)

In [ ]:
def lr_schedule(epoch):
    if epoch < warmup_epochs:
        return float(epoch + 1) / float(warmup_epochs)
    progress = float(epoch - warmup_epochs) / float(max(1, epochs - warmup_epochs))
    return 0.5 * (1. + math.cos(math.pi * progress))

scheduler = LambdaLR(optimizer, lr_lambda=lr_schedule)

def mixup_data(x, y, alpha=mixup_alpha):
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1.0
    index = torch.randperm(x.size(0), device=x.device)
    mixed_x = lam * x + (1 - lam) * x[index]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

In [ ]:
def validate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            running_loss += loss.item() * inputs.size(0)
            _, preds = outputs.max(1)
            correct += preds.eq(targets).sum().item()
            total += targets.size(0)

    return running_loss / total, correct / total

In [ ]:
class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0, path='area.pth'):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.inf
        self.delta = delta
        self.path = path
        
    def __call__(self, val_loss, model):
        score = -val_loss
        
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0
            
    def save_checkpoint(self, val_loss, model):
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}). Saving model...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

def train_model(model, train_loader, val_loader, test_loader, device, 
                backbone_lr=1e-5, classifier_lr=1e-4, weight_decay=1e-2,
                gradient_accumulation_steps=2, patience=5):
    
    optimizer = optim.AdamW([
        {'params': [p for n, p in model.named_parameters() if 'classifier' not in n], 'lr': backbone_lr},
        {'params': model.classifier.parameters(), 'lr': classifier_lr}
    ], weight_decay=weight_decay)
    
    scheduler = LambdaLR(optimizer, lr_lambda=lr_schedule)
    early_stopping = EarlyStopping(patience=patience, verbose=True)

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0.0
        total = 0
        optimizer.zero_grad()
        
        pbar = tqdm(enumerate(train_loader), total=len(train_loader), 
                    desc=f"Epoch {epoch+1}/{epochs}")
        for batch_idx, (inputs, targets) in pbar:
            inputs, targets = inputs.to(device), targets.to(device)
            
            if mixup_alpha > 0:
                inputs, targets_a, targets_b, lam = mixup_data(inputs, targets)
                outputs = model(inputs)
                loss = lam * criterion(outputs, targets_a) + \
                      (1 - lam) * criterion(outputs, targets_b)
            else:
                outputs = model(inputs)
                loss = criterion(outputs, targets)

            loss = loss / gradient_accumulation_steps
            loss.backward()

            if (batch_idx + 1) % gradient_accumulation_steps == 0 or batch_idx == len(train_loader) - 1:
                optimizer.step()
                optimizer.zero_grad()
                scheduler.step()

            running_loss += loss.item() * gradient_accumulation_steps * inputs.size(0)
            _, preds = outputs.max(1)
            if mixup_alpha > 0:
                correct += lam * preds.eq(targets_a).sum().item() + \
                          (1 - lam) * preds.eq(targets_b).sum().item()
            else:
                correct += preds.eq(targets).sum().item()
            total += targets.size(0)

            pbar.set_postfix({
                'loss': running_loss / total,
                'acc': 100. * correct / total
            })

        train_loss = running_loss / total
        train_acc = correct / total
        val_loss, val_acc = validate(model, val_loader, criterion, device)

        print(f'\nEpoch {epoch+1}/{epochs}')
        print(f'Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}')
        print(f'Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}')

        early_stopping(val_loss, model)
        if early_stopping.early_stop:
            print("Early stopping triggered!")
            break

    model.load_state_dict(torch.load('area.pth'))
    return model


In [ ]:
config = {
    'backbone_lr': 1e-4,
    'classifier_lr': 1e-4,
    'weight_decay': 1e-2,
    'gradient_accumulation_steps': 2,
    'patience': 5
}

model = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    device=device,
    **config
)

#### Now we save the predicted Region_IDs for train, val and test so that they can be used for training LatLong prredictions (in case of train and val) and testing LatLong

In [ ]:
def predict_and_save(checkpoint_path, model, dataloader, output_csv, device):

    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.to(device)
    model.eval()

    preds = []
    correct = 0
    total = 0
    sample_id = 0

    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs = inputs.to(device)
            targets = targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(dim=1)

            correct += (predicted == targets).sum().item()
            total += targets.size(0)

            for p in predicted.cpu().numpy():
                preds.append((sample_id, int(p))) 
                sample_id += 1

    with open(output_csv, mode='w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['id', 'Region_ID'])
        writer.writerows(preds)

    accuracy = correct / total
    print(f"Saved {len(preds)} predictions to {output_csv}")
    print(f"Validation Accuracy: {accuracy:.4f}")

In [ ]:
train_dataset = CustomImageDataset(train_csv, train_img_dir, transform=train_transforms)

train_loader = DataLoader(train_dataset, batch_size=batch_size,
                         shuffle=False, num_workers=num_workers,
                         pin_memory=True)

predict_and_save(
    checkpoint_path="area.pth",
    model=model,
    dataloader=train_loader,
    output_csv="area_predictions_train.csv",
    device=device
)

In [ ]:
predict_and_save(
    checkpoint_path="area.pth",
    model=model,
    dataloader=val_loader,
    output_csv="area_predictions_val.csv",
    device=device
)

In [ ]:
import pandas as pd

df = pd.read_csv('area_predictions_train.csv')  

df['Region_ID'] = df['Region_ID'] + 1  

df.to_csv('area_predictions_train.csv', index=False)  

In [ ]:
class TestImageDataset(Dataset):
    def __init__(self, img_dir, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        self.image_files = sorted(os.listdir(img_dir)) 

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.image_files[idx])
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, 0 

In [ ]:
test_img_dir = "/data3/pratyush.jena/misc/MMT/SMAI_Project/Phase_2_data/Phase_2_data/images_test"
test_dataset = TestImageDataset(test_img_dir, transform=val_transforms)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)

In [ ]:
def predict_test_and_save(checkpoint_path, model, dataloader, output_csv, device):
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.to(device)
    model.eval()

    if os.path.exists(output_csv):
        existing_df = pd.read_csv(output_csv)
        start_id = existing_df['id'].max() + 1
    else:
        start_id = 0

    preds = []
    sample_id = start_id

    with torch.no_grad():
        for inputs, _ in dataloader:  
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(dim=1)
            for p in predicted.cpu().numpy():
                preds.append((sample_id+369, int(p)))
                sample_id += 1

    with open(output_csv, mode='a', newline='') as f:
        writer = csv.writer(f)
        if start_id == 0:  
            writer.writerow(['id', 'Region_ID'])
        writer.writerows(preds)

    print(f"Appended {len(preds)} test predictions to {output_csv}")

In [ ]:
predict_test_and_save(
    checkpoint_path="area.pth",
    model=model,
    dataloader=test_loader,
    output_csv="area_predictions_test.csv",
    device=device
)

In [ ]:
import pandas as pd 
df = pd.read_csv('area_predictions_test.csv')
df['Region_ID'] += 1
df.to_csv('area_predictions_test.csv', index=False)

In [ ]:

import pandas as pd

labels_df = pd.read_csv('/data3/pratyush.jena/misc/MMT/SMAI_Project/Phase_2_data/Phase_2_data/labels_val.csv')
predictions_df = pd.read_csv('/data3/pratyush.jena/misc/MMT/SMAI_Project/area_predictions_val.csv')

labels_df['id'] = labels_df['filename'].str.extract(r'img_(\d+)\..*', expand=False).astype(int)

merged_df = pd.merge(
    labels_df,
    predictions_df,
    on='id',
    how='inner',
    suffixes=('_original', '_predicted')  
)

result_df = merged_df[['filename', 'latitude', 'longitude', 'Region_ID_predicted']].rename(
    columns={'Region_ID_predicted': 'Region_ID'}
)

# Save to new CSV
result_df.to_csv('/data3/pratyush.jena/misc/MMT/SMAI_Project/area_predictions_val.csv', index=False)


In [ ]:
labels_df = pd.read_csv('/data3/pratyush.jena/misc/MMT/SMAI_Project/Phase_2_data/Phase_2_data/labels_train.csv')
predictions_df = pd.read_csv('/data3/pratyush.jena/misc/MMT/SMAI_Project/area_predictions_train.csv')

labels_df['id'] = labels_df['filename'].str.extract(r'img_(\d+)\..*', expand=False).astype(int)

merged_df = pd.merge(
    labels_df,
    predictions_df,
    on='id',
    how='inner',
    suffixes=('_original', '_predicted')  
)

result_df = merged_df[['filename', 'latitude', 'longitude', 'Region_ID_predicted']].rename(
    columns={'Region_ID_predicted': 'Region_ID'}
)

result_df.to_csv('/data3/pratyush.jena/misc/MMT/SMAI_Project/area_predictions_train.csv', index=False)